In [141]:
# M.Duchnowski
# Upskill FY2022 : Data Science Academy
# Week 5 Homework
# 11/30/2021

In [142]:
import numpy as np
import pandas as pd
import nltk 

In [143]:
#
# Part 1 : Create action sequence variable ("ActSeq") and time interval sequence ("Elapsed") 
#

df = pd.read_csv (r'PSTRE_syntheticData.csv', sep='\t', )

#Sort data, for good measure
df.sort_values(by=['TestTakerID','Timestamp'], inplace=True)

# Get Acion Sequence 'ActSeq' using a zero-based group by function
# https://stackoverflow.com/questions/56963520/creating-sequential-id-in-panda-dataframe-for-given-column-values
df["ActSeq"] = df.groupby(['TestTakerID']).cumcount()

# Extract new variable 'Elapsed', the difference between Timestamp values 
x = df['Timestamp'].diff(periods=1).to_frame().rename(columns={"Timestamp": "Elapsed"})
x.loc[(x['Elapsed'] <= 0), 'Elapsed'] = np.nan

# Merge 'Elapsed' back to orginal data
df = df.join(x)
df.head(25)


,TestTakerID,Timestamp,Coding,ActSeq,Elapsed
0,1001042,0,START,0,NaN
1,1001042,44985,SS_Se,1,44985.0
2,1001042,55852,SS_Se_OK,2,10867.0
3,1001042,55950,SS_Type_2,3,98.0
4,1001042,66546,E,4,10596.0
5,1001042,90189,SS,5,23643.0
6,1001042,96096,E,6,5907.0
7,1001042,101813,Next,7,5717.0
8,1001042,105379,Next_OK,8,3566.0
9,1001042,105382,END,9,3.0


In [108]:
#
# Part 2 : Create Time to First Action 
#

dfFirstAction = df[df['ActSeq'] == 1].reset_index(drop=True).drop(['ActSeq', 'Elapsed'], axis=1).rename(columns={"Timestamp": "TimeFirstAction", "Coding" : "FirstAction"})
dfFirstAction

,TestTakerID,TimeFirstAction,FirstAction
0,1001042,44985,SS_Se
1,1001079,21234,SS_So
2,1001103,39002,SS_Se
3,1001112,54895,E
4,1002087,158089,SS_Se_OK
...,...,...,...
1074,3464227,103256,SS_Save
1075,3491220,115314,E
1076,4044376,110214,Next
1077,4049370,59622,SS_Save


In [144]:
#
# Part 3 : Create Bigrams for process data assessment path 
#

#Build a single string
# https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
dfPaths = df.groupby(['TestTakerID'])['Coding'].apply(' '.join).reset_index().rename(columns={"Coding": "Path"})

#Extract bigrams from Path and generate Bigrams 
dfTokens = dfPaths['Path'].apply(lambda row: list(nltk.bigrams(row.split(' ')))).to_frame().rename(columns={"Path": "Bigrams"})

# Merge back to orginal data
dfBigrams = dfPaths.join(dfTokens)

dfBigrams.head(25)

,TestTakerID,Path,Bigrams
0,1001042,START SS_Se SS_Se_OK SS_Type_2 E SS E Next Nex...,"[(START, SS_Se), (SS_Se, SS_Se_OK), (SS_Se_OK,..."
1,1001079,START SS_So SS_So_1B SS_So_OK SS E SS E SS E S...,"[(START, SS_So), (SS_So, SS_So_1B), (SS_So_1B,..."
2,1001103,START SS_Se SS_Se_OK SS_Type_2 E SS E SS E Nex...,"[(START, SS_Se), (SS_Se, SS_Se_OK), (SS_Se_OK,..."
3,1001112,START E SS E Next Next_OK END,"[(START, E), (E, SS), (SS, E), (E, Next), (Nex..."
4,1002087,START SS_Se_OK SS_Type_200 SS_So_1B SS_So_OK E...,"[(START, SS_Se_OK), (SS_Se_OK, SS_Type_200), (..."
5,1002110,START E SS E E_S Next Next_OK END,"[(START, E), (E, SS), (SS, E), (E, E_S), (E_S,..."
6,1003077,START SS_So SS_So_1B SS_So_OK E Next Next_OK END,"[(START, SS_So), (SS_So, SS_So_1B), (SS_So_1B,..."
7,1003110,START Next Next_C Next Next_C E Next Next_C E ...,"[(START, Next), (Next, Next_C), (Next_C, Next)..."
8,1003147,START Next Next_OK END,"[(START, Next), (Next, Next_OK), (Next_OK, END)]"
9,1003165,START SS_So_1B SS_So_OK SS_So_OK SS_Save E E_S...,"[(START, SS_So_1B), (SS_So_1B, SS_So_OK), (SS_..."
